## Tokenización y Procesamiento de Texto en Spark MLlib

El procesamiento de texto en Spark MLlib se usa para analizar y extraer información clave de textos. Uno de los primeros pasos es la tokenización, que consiste en dividir un texto en palabras individuales o "tokens".

#### ¿Por qué es útil la tokenización?

- Permite analizar palabras más usadas.
- Facilita la limpieza del texto (eliminación de signos, palabras vacías, etc.).
- Es un paso clave en modelos de Machine Learning con texto (Ej. NLP).


In [3]:
import org.apache.spark.ml.feature.{Tokenizer, StopWordsRemover}
import org.apache.spark.sql.functions._

// 1)  Crear sesión de Spark
val spark = SparkSession.builder()
  .appName("Tokenizacion Job Analysis")
  .master("local[*]") // Importante para ejecutar en local
  .getOrCreate()

// 2) argar los datos desde un CSV
val df = spark.read.option("header", "true").option("inferSchema", "true").csv("job.csv")


// 3) Ver las primeras filas
df.show(5)

// 4) Tokenizar el texto
val tokenizer = new Tokenizer()
  .setInputCol("job_details")
  .setOutputCol("words")

val tokenizedDF = tokenizer.transform(df)

// 5) Remover palabras comunes (stopwords)
val remover = new StopWordsRemover()
  .setInputCol("words")
  .setOutputCol("filtered_words")

val df_filtered = remover.transform(tokenizedDF)

// Mostrar algunas palabras filtradas
df_filtered.select("filtered_words").show(5, truncate = false)

+------+--------------------+
|job_ID|         job_details|
+------+--------------------+
|     1|We are looking fo...|
|     2|Join our team as ...|
|     3|Seeking a product...|
|     4|Looking for a bac...|
+------+--------------------+

+-------------------------------------------------------------------------------------+
|filtered_words                                                                       |
+-------------------------------------------------------------------------------------+
|[looking, data, scientist, experience, python, machine, learning.]                   |
|[join, team, software, engineer, specialized, cloud, computing, devops.]             |
|[seeking, product, manager, strong, background, ux/ui, design, agile, methodologies.]|
|[looking, backend, developer, expertise, java,, spring, boot,, microservices.]       |
+-------------------------------------------------------------------------------------+



spark = org.apache.spark.sql.SparkSession@1f77ca8d
df = [job_ID: int, job_details: string]
tokenizer = tok_0da4cd4419a0
tokenizedDF = [job_ID: int, job_details: string ... 1 more field]
remover = StopWordsRemover: uid=stopWords_95b14ec5e7bc, numStopWords=181, locale=en_US, caseSensitive=false
df_filtered = [job_ID: int, job_details: string ... 2 more fields]


[job_ID: int, job_details: string ... 2 more fields]

#### Explicación Paso a Paso

**1.** Se crea una sesión de Spark usando SparkSession.builder().

**2.** Se carga el archivo CSV que contiene las ofertas de empleo.

**3.** Se muestran las primeras filas para verificar los datos.

**4.** Se usa Tokenizer para separar las palabras en cada descripción (job_details).

**5.** Se aplica StopWordsRemover para eliminar palabras comunes como "the", "and", "in", etc.

**6.** Se muestra la columna filtered_words con las palabras clave más relevantes.


In [4]:
// 6) Convertir las listas de palabras en filas individuales
val wordsDF = tokenizedDF
  .select(explode(col("words")).alias("word"))
  .groupBy("word")
  .count()
  .orderBy(desc("count"))

// 7) Mostrar las palabras más usadas
wordsDF.show(10)

+---------+-----+
|     word|count|
+---------+-----+
|        a|    5|
|       in|    4|
|      and|    4|
|     with|    3|
|      for|    2|
|  looking|    2|
|expertise|    1|
|  devops.|    1|
|developer|    1|
|    java,|    1|
+---------+-----+
only showing top 10 rows



wordsDF = [word: string, count: bigint]


[word: string, count: bigint]